Importing necessary libraries

In [1]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity

Preprocessing the Data

In [2]:
df=pd.read_csv("Cognifyz Internship Project Dataset.csv")
df.describe()

,Restaurant ID,Country Code,Longitude,Latitude,Average Cost for two,Price range,Aggregate rating,Votes
count,9.551000e+03,9551.000000,9551.000000,9551.000000,9551.000000,9551.000000,9551.000000,9551.000000
mean,9.051128e+06,18.365616,64.126574,25.854381,1199.210763,1.804837,2.666370,156.909748
std,8.791521e+06,56.750546,41.467058,11.007935,16121.183073,0.905609,1.516378,430.169145
min,5.300000e+01,1.000000,-157.948486,-41.330428,0.000000,1.000000,0.000000,0.000000
25%,3.019625e+05,1.000000,77.081343,28.478713,250.000000,1.000000,2.500000,5.000000
50%,6.004089e+06,1.000000,77.191964,28.570469,400.000000,2.000000,3.200000,31.000000
75%,1.835229e+07,1.000000,77.282006,28.642758,700.000000,2.000000,3.700000,131.000000
max,1.850065e+07,216.000000,174.832089,55.976980,800000.000000,4.000000,4.900000,10934.000000


In [3]:
df = df.drop(['Restaurant ID', 'Restaurant Name', 'Address', 'Currency','Rating color', 'Rating text', 'Switch to order menu', 'Locality', 'Locality Verbose'],axis=1)

In [4]:
df['Has Table booking'] = df['Has Table booking'].map({'Yes': 1, 'No': 0})
df['Has Online delivery'] = df['Has Online delivery'].map({'Yes': 1, 'No': 0})
df['Is delivering now'] = df['Is delivering now'].map({'Yes': 1, 'No': 0})


In [5]:
#To remove zero ratings 
df = df[df['Aggregate rating'] > 0]

In [6]:
#Checking which columns contain categorical data 
df.select_dtypes(include='object').columns

Index(['City', 'Cuisines'], dtype='object')

In [7]:
df = pd.get_dummies(df, columns=['City', 'Cuisines'], drop_first=True)

Determining the criteria

In [8]:
user_pref = {'Price range': 2,'Has Online delivery': 1,'Has Table booking': 1,'Cuisines_Italian': 1,'Cuisines_North Indian': 0,'Cuisines_Chinese': 0}

Implementing content based filtering

In [9]:
df_user_pref = pd.DataFrame([user_pref])

for col in df.columns:
    if col not in df_user_pref.columns and col != 'Aggregate rating':
        df_user_pref[col] = 0

# Reorder columns to match training data
df_user_pref = df_user_pref[df.drop('Aggregate rating', axis=1).columns]

# Computing cosine similarity between user preferences and all restaurants
similarity = cosine_similarity(df_user_pref, df.drop('Aggregate rating', axis=1))

# Get top recommendations
top_recomm = similarity[0].argsort()[::-1][:10]  

# Display recommended restaurants
recommended_restaurants = df.iloc[top_recomm]
print(recommended_restaurants[['Aggregate rating', 'Price range', 'Has Table booking', 'Has Online delivery']])


C:\Users\Yash\AppData\Local\Temp\ipykernel_2748\2704241394.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_user_pref[col] = 0
C:\Users\Yash\AppData\Local\Temp\ipykernel_2748\2704241394.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_user_pref[col] = 0
C:\Users\Yash\AppData\Local\Temp\ipykernel_2748\2704241394.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axi

    Aggregate rating  Price range  Has Table booking  Has Online delivery
33               3.6            3                  0                    0
25               3.8            3                  0                    0
47               4.8            3                  0                    0
44               4.7            3                  0                    0
36               3.6            4                  0                    0
32               3.1            4                  0                    0
65               4.1            4                  0                    0
79               4.4            4                  0                    0
43               4.8            4                  0                    0
34               4.3            4                  0                    0


Evaluating the quality of recommendations

In [11]:
recommended_restaurants['Similarity'] = similarity[0][top_recomm]

print("Average rating of recommendations:", recommended_restaurants['Aggregate rating'].mean())
print("Top similarity scores:\n", recommended_restaurants['Similarity'].head(10))

Average rating of recommendations: 4.119999999999999
Top similarity scores:
 33    0.026822
25    0.026775
47    0.026605
44    0.026221
36    0.026037
32    0.025971
65    0.025670
79    0.025443
43    0.025421
34    0.025272
Name: Similarity, dtype: float64


C:\Users\Yash\AppData\Local\Temp\ipykernel_2748\2148024261.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recommended_restaurants['Similarity'] = similarity[0][top_recomm]
